In [1]:
import pandas as pd
import numpy as np
import keras 
import tensorflow as tf
from sklearn.datasets import load_wine

In [2]:
from tensorflow.python.client import device_lib

In [45]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18001086060666618289,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14361195058238842
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8247341632614632115
 physical_device_desc: "device: XLA_GPU device"]

In [46]:
tf.device("XLA_GPU:0")

In [5]:
data = load_wine()

In [6]:
data1 = pd.DataFrame(data.data,columns = data.feature_names)

In [7]:
data1['target'] = data.target

In [8]:
df = data1

In [9]:
data1.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


In [10]:
cat_features=[i for i in data1.columns if data1.dtypes[i]=='object']
cat_features

[]

In [37]:
from keras.models import Sequential
from keras.layers import Dense , LeakyReLU , ELU , Dropout ,Flatten , BatchNormalization , Activation
from keras.activations import relu , sigmoid

In [12]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X = df.iloc[:,0:13]
y = df.iloc[:,13]

In [15]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.3, random_state=42)

In [17]:
from sklearn.preprocessing import StandardScaler

In [18]:
sc = StandardScaler()

In [19]:
X_train =  sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

**Creating Hyper parameter tuning !**

In [50]:
def hyper_tune_model(layers,activation):
    model = Sequential()
    for i , nodes in enumerate(layers):
        if i == 0:
            model.add(Dense(nodes,input_dim = X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
    model.add(Dense(units=1,kernel_initializer = 'glorot_uniform',activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [60]:
model = KerasClassifier(build_fn = hyper_tune_model,verbose=1)

In [61]:
layers = [(20),(40,20)]
activation = ['sigmoid','relu']

In [68]:
parameter_grid = dict(layers=layers,activation=activation,batch_size=(128,256),epochs=(10,30))

In [69]:
parameter_grid

{'layers': [20, (40, 20)],
 'activation': ['sigmoid', 'relu'],
 'batch_size': (128, 256),
 'epochs': (10, 30)}

In [70]:
grid = GridSearchCV(estimator = model , param_grid=parameter_grid,cv=5)

In [71]:
grid_result = grid.fit(X_train,y_train)

d:\program files\python3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "d:\program files\python3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Akhil Sanker\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\wrappers\scikit_learn.py", line 223, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "C:\Users\Akhil Sanker\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\wrappers\scikit_learn.py", line 157, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
  File "<ipython-input-50-7a2e3d9a1948>", line 3, in hyper_tune_model
    for i , nodes in enumerate(layers):
TypeError: 'int' object is not iterable

  FitFai

Epoch 1/10
1/1 [==============================] - 0s 7ms/step - loss: 1.0444 - accuracy: 0.2727
Epoch 2/10
1/1 [==============================] - 0s 6ms/step - loss: 1.0068 - accuracy: 0.2727
Epoch 3/10
1/1 [==============================] - 0s 6ms/step - loss: 0.9722 - accuracy: 0.3131
Epoch 4/10
1/1 [==============================] - 0s 6ms/step - loss: 0.9123 - accuracy: 0.3434
Epoch 5/10
1/1 [==============================] - 0s 7ms/step - loss: 0.8837 - accuracy: 0.3535
Epoch 6/10
1/1 [==============================] - 0s 7ms/step - loss: 0.8016 - accuracy: 0.3535
Epoch 7/10
1/1 [==============================] - 0s 4ms/step - loss: 0.8439 - accuracy: 0.3535
Epoch 8/10
1/1 [==============================] - 0s 7ms/step - loss: 0.8479 - accuracy: 0.3131
Epoch 9/10
1/1 [==============================] - 0s 7ms/step - loss: 0.8061 - accuracy: 0.3636
Epoch 10/10
1/1 [==============================] - 0s 4ms/step - loss: 0.7369 - accuracy: 0.4400
Epoch 1/10
1/1 [=======================

In [72]:
print(grid_result.best_score_,grid_result.best_params_)

0.6446666717529297 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': (40, 20)}


In [73]:
pred_y  = grid.predict(X_test
                      )

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
1/1 [==============================] - 0s 3ms/step


In [75]:
y_pred = (pred_y>0.5)

In [77]:
from sklearn.metrics import confusion_matrix , accuracy_scorecy_score

In [78]:
cm = confusion_matrix(y_pred,y_test)

In [79]:
score = accuracy_score(y_pred,y_test)

In [80]:
cm , score

(array([[13,  0,  0],
        [ 6, 21, 14],
        [ 0,  0,  0]], dtype=int64),
 0.6296296296296297)